In [8]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
print(tokenizer("Hello world")['input_ids'])
print(tokenizer(" Hello world"))

[15496, 995]
{'input_ids': [18435, 995], 'attention_mask': [1, 1]}


In [13]:
import os
file = '../data/jotun 02-05-2023/script.txt'

with open(file, 'r') as f:
    text = f.read()

chunks = text.split('\n\n')
tsizes = [len(tokenizer(chunk)['input_ids']) for chunk in chunks]

In [24]:
posts = []
bag = []
total_tokens = 0
max_size = 3600
for chunk, tsize in zip(chunks, tsizes):
    if total_tokens + tsize > max_size:
        posts.append(bag)
        bag = []
        total_tokens = 0
    total_tokens += tsize
    bag.append(chunk)
else:
    posts.append(bag)

In [20]:
len(posts)

11

In [26]:
for i, post in enumerate(posts):
    print(len(post))
    post_text = '\n\n'.join(post)

    with open(f"./posts/post_{i}.txt", 'w') as f:
        f.write(post_text)

162
143
152
166
153
150
153
117
207
161
110
